In [3]:
from pathlib import Path
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv

DATA_ROOT = Path('/content/drive/MyDrive/mit-bih-arrhythmia-database-1.0.0')
OUTPUT_IMG_DIR = Path('/content/drive/MyDrive/mitbih_images_aami')
OUTPUT_IMG_DIR.mkdir(parents=True, exist_ok=True)

MITBIH_TO_AAMI = {
    'N':'N','L':'N','R':'N','e':'N','j':'N',
    'A':'S','a':'S','J':'S','S':'S',
    'V':'V','E':'V',
    'F':'F',
    '/':'Q','f':'Q','Q':'Q','x':'Q','?':'Q'
}

def map_symbol_to_aami(sym):
    return MITBIH_TO_AAMI.get(sym, 'Q')

def read_record(record_path_no_ext):
    record = wfdb.rdrecord(str(record_path_no_ext))
    ann = wfdb.rdann(str(record_path_no_ext), 'atr')
    sig = record.p_signal
    fs = record.fs
    return sig, ann, fs

WINDOW_PRE_SEC = 0.2
WINDOW_POST_SEC = 0.6

IMG_ROOT = OUTPUT_IMG_DIR
for c in ['N','S','V','F','Q']:
    (IMG_ROOT / c).mkdir(parents=True, exist_ok=True)

meta_csv = IMG_ROOT / "metadata.csv"

def save_ecg_segment_image(signal1d, out_path, dpi=100, grid=True):
    if out_path.exists():
        return
    plt.figure(figsize=(3,3), dpi=dpi)
    ax = plt.gca()
    ax.plot(signal1d, linewidth=1)
    ax.set_xticks([])
    ax.set_yticks([])
    if grid:
        ax.grid(True, linewidth=0.3, linestyle='--')
    plt.margins(0.05)
    plt.savefig(str(out_path), bbox_inches='tight', pad_inches=0.02)
    plt.close()

# Load existing metadata
existing_rows = []
if meta_csv.exists():
    with open(meta_csv, 'r') as f:
        reader = csv.reader(f)
        next(reader)
        existing_rows = [row for row in reader]
existing_files = set(row[0] for row in existing_rows)
rows = existing_rows.copy()

records = sorted([p.stem for p in DATA_ROOT.glob("*.dat")])
print("Found records:", records)

# -----------------------------
# BATCH PROCESSING SETTINGS
# -----------------------------
BATCH_SIZE = 5  # number of records per batch
start_batch = 5  # start batch index
end_batch = min(BATCH_SIZE, len(records))  # end batch index

records_batch = records[start_batch:end_batch]
print(f"Processing records {start_batch} to {end_batch-1}")

# Process one batch
for rec in tqdm(records_batch, desc="Processing batch"):
    try:
        sig, ann, fs = read_record(DATA_ROOT / rec)
    except Exception as e:
        print("Error reading", rec, e)
        continue
    lead = sig[:,0]
    pre = int(WINDOW_PRE_SEC * fs)
    post = int(WINDOW_POST_SEC * fs)
    for i, sample in enumerate(ann.sample):
        sym = ann.symbol[i]
        aami = map_symbol_to_aami(sym)
        fname = f"{rec}_{sample}_{i}.png"
        if fname in existing_files:
            continue
        start = sample - pre
        end = sample + post
        if start < 0 or end > len(lead):
            continue
        segment = lead[start:end]
        seg = (segment - np.mean(segment)) / (np.std(segment) + 1e-8)
        out_path = IMG_ROOT / aami / fname
        save_ecg_segment_image(seg, out_path, dpi=100, grid=False)
        # save metadata after each image
        rows.append([fname, rec, sample, sym, aami])
        with open(meta_csv, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['filename','record','sample','mitbih_symbol','aami'])
            writer.writerows(rows)

print(f"Batch {start_batch}-{end_batch-1} finished. You can run next batch.")


Found records: ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124', '200', '201', '202', '203', '205', '207', '208', '209', '210', '212', '213', '214', '215', '217', '219', '220', '221', '222', '223', '228', '230', '231', '232', '233', '234']
Processing records 5 to 4


Processing batch: 0it [00:00, ?it/s]

Batch 5-4 finished. You can run next batch.


In [ ]:
from pathlib import Path

base = Path("/content/drive/MyDrive/mitbih_images_aami")

for cls in ["N", "S", "V", "F", "Q"]:
    folder = base / cls
    count = len(list(folder.glob("*.png")))
    print(f"{cls}: {count}")


N: 10000
S: 2730
V: 5000
F: 802
Q: 5000


In [ ]:
import os
import shutil
import random
from pathlib import Path

SOURCE_DIR = Path("/content/drive/MyDrive/mitbih_images_aami")
DEST_DIR   = Path("/content/drive/MyDrive/mitbih_images_reduced")

TARGET_COUNTS = {
    "N": 5000,
    "S": 2000,
    "V": 3000,
    "F": 800,
    "Q": 3000
}

for cls in TARGET_COUNTS.keys():
    (DEST_DIR / cls).mkdir(parents=True, exist_ok=True)

def reduce_class(cls, target_count):
    all_images = list((SOURCE_DIR / cls).glob("*.png"))
    random.shuffle(all_images)

    selected = all_images[:target_count]

    for img in selected:
        shutil.copy(img, DEST_DIR / cls)

    print(f"{cls}: Took {len(selected)} images")

for cls, count in TARGET_COUNTS.items():
    reduce_class(cls, count)

print("Dataset reduction complete!")

N: Took 5000 images
S: Took 2000 images
V: Took 3000 images
F: Took 800 images
Q: Took 3000 images
Dataset reduction complete!


In [ ]:
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split

DATA_DIR = Path("/content/drive/MyDrive/mitbih_images_reduced")
OUTPUT_DIR = Path("/content/drive/MyDrive/mitbih_split_small")

for split in ["train", "val", "test"]:
    for cls in ["N","S","V","F","Q"]:
        (OUTPUT_DIR / split / cls).mkdir(parents=True, exist_ok=True)

def split_class(cls):
    images = list((DATA_DIR / cls).glob("*.png"))
    images = [str(p) for p in images]

    train_imgs, temp = train_test_split(images, test_size=0.30, random_state=42)
    val_imgs, test_imgs = train_test_split(temp, test_size=0.50, random_state=42)

    for img in train_imgs:
        shutil.copy(img, OUTPUT_DIR / "train" / cls)

    for img in val_imgs:
        shutil.copy(img, OUTPUT_DIR / "val" / cls)

    for img in test_imgs:
        shutil.copy(img, OUTPUT_DIR / "test" / cls)

    print(f"{cls}: Train={len(train_imgs)}, Val={len(val_imgs)}, Test={len(test_imgs)}")

for cls in ["N","S","V","F","Q"]:
    split_class(cls)

print("Split complete!")

N: Train=3500, Val=750, Test=750
S: Train=1400, Val=300, Test=300
V: Train=2100, Val=450, Test=450
F: Train=560, Val=120, Test=120
Q: Train=2100, Val=450, Test=450
Split complete!


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

TRAIN_DIR = "/content/drive/MyDrive/mitbih_split_small/train"
VAL_DIR   = "/content/drive/MyDrive/mitbih_split_small/val"
TEST_DIR  = "/content/drive/MyDrive/mitbih_split_small/test"

In [5]:
train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.10,
    brightness_range=[0.8, 1.2]
)

val_aug = ImageDataGenerator(rescale=1./255)
test_aug = ImageDataGenerator(rescale=1./255)

In [6]:
BATCH = 32
IMG_SIZE = (224, 224)

train_data = train_aug.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical"
)

val_data = val_aug.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical"
)

test_data = test_aug.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode="categorical",
    shuffle=False
)

Found 9660 images belonging to 5 classes.
Found 2070 images belonging to 5 classes.
Found 2070 images belonging to 5 classes.


In [13]:
from tensorflow.keras import layers, models

def build_model(num_classes):
    model = models.Sequential([
        layers.Input(shape=(224, 224, 3)),

        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D(),

        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(),

        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.MaxPooling2D(),

        layers.Flatten(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation="softmax")
    ])

    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    model.build(input_shape=(None, 224, 224, 3))
    model.summary()

    return model


In [ ]:
import os
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ECG.keras"

if os.path.exists(checkpoint_path):
    print("🔄 Found previous model — resuming training...")
    model = tf.keras.models.load_model(checkpoint_path)
else:
    print("🆕 No previous checkpoint found — creating new model...")
    model = build_model(num_classes=5)


🔄 Found previous model — resuming training...


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),

    ModelCheckpoint(
        filepath=checkpoint_path,
        save_best_only=True,
        monitor='val_loss',
        verbose=1
    )
]

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.3423 - loss: 2.0472
Epoch 1: val_loss improved from inf to 1.47918, saving model to /content/drive/MyDrive/Colab Notebooks/ECG.keras
302/302 ━━━━━━━━━━━━━━━━━━━━ 1456s 5s/step - accuracy: 0.3424 - loss: 2.0457 - val_accuracy: 0.3623 - val_loss: 1.4792
Epoch 2/20
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5414 - loss: 1.1526
Epoch 2: val_loss improved from 1.47918 to 0.46884, saving model to /content/drive/MyDrive/Colab Notebooks/ECG.keras
302/302 ━━━━━━━━━━━━━━━━━━━━ 1422s 5s/step - accuracy: 0.5418 - loss: 1.1519 - val_accuracy: 0.8333 - val_loss: 0.4688
Epoch 3/20
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.8004 - loss: 0.5950
Epoch 3: val_loss improved from 0.46884 to 0.32983, saving model to /content/drive/MyDrive/Colab Notebooks/ECG.keras
302/302 ━━━━━━━━━━━━━━━━━━━━ 1420s 5s/step - accuracy: 0.8004 - loss: 0.5949 - val_accuracy: 0.8855 - val_loss: 0.3298
Epoch 4/20
302/302 ━━━━━━━━━━━━━━━━━━━━ 0